In [2]:
#import the libraries
from __future__ import print_function
import sys
import cv2
from random import randint

In [3]:
trackerTypes = ['BOOSTING', 'MIL', 'KCF','TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']

In [54]:
def createTrackerByName(trackerType):
  # Create a tracker based on tracker name
  if trackerType == trackerTypes[0]:
    tracker = cv2.legacy.TrackerBoosting_create()
  elif trackerType == trackerTypes[1]: 
    tracker = cv2.TrackerMIL_create()
  elif trackerType == trackerTypes[2]:
    tracker = cv2.TrackerKCF_create()
  elif trackerType == trackerTypes[3]:
    tracker = cv2.legacy.TrackerTLD_create()
  elif trackerType == trackerTypes[4]:
    tracker = cv2.legacy.TrackerMedianFlow_create()
  elif trackerType == trackerTypes[5]:
    tracker = cv2.legacy.TrackerGOTURN_create()
  elif trackerType == trackerTypes[6]:
    tracker = cv2.legacy.TrackerMOSSE_create()
  elif trackerType == trackerTypes[7]:
    tracker = cv2.legacy.TrackerCSRT_create()
  else:
    tracker = None
    print('Incorrect tracker name')
    print('Available trackers are:')
    for t in trackerTypes:
      print(t)
    
  return tracker

In [50]:
print("Default tracking algoritm is CSRT \n"
      "Available tracking algorithms are:\n")
for t in trackerTypes:
    print(t)      

Default tracking algoritm is CSRT 
Available tracking algorithms are:

BOOSTING
MIL
KCF
TLD
MEDIANFLOW
GOTURN
MOSSE
CSRT


In [51]:
!wget https://chateaut.fr/documents/Bean_clip.avi

--2023-01-16 12:52:53--  https://chateaut.fr/documents/Bean_clip.avi
Resolving chateaut.fr (chateaut.fr)... 213.186.33.24
Connecting to chateaut.fr (chateaut.fr)|213.186.33.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 561913 (549K) [video/x-msvideo]
Saving to: ‘Bean_clip.avi.2’

Bean_clip.avi.2     100%[===================>] 548.74K  1.44MB/s    in 0.4s    

2023-01-16 12:52:54 (1.44 MB/s) - ‘Bean_clip.avi.2’ saved [561913/561913]



In [63]:
trackerType = "CSRT"    

In [64]:
tracker=createTrackerByName(trackerType)

In [65]:
# Init video reader and writer
cap = cv2.VideoCapture('Bean_clip.avi')
#
#Initializing a bounding box 
#bbox = img
  ## Bean Tracking
#bbox =  (416, 206, 128, 124) # Children
#bbox =  (379, 33, 89, 106)   # Bean 
bbox =  (114, 99, 125, 100)  # Women
#
res=(int(cap.get(3)),int(cap.get(4))) #resolution
fourcc = cv2.VideoWriter_fourcc(*'MP4V') #codec
!rm output.mp4
out = cv2.VideoWriter('output.mp4', fourcc, 10.0, res)
# Init Tracker
status, frame = cap.read()
ok = tracker.init(frame, bbox)
while(1):
    status, frame = cap.read()
    if status==False:
      break
    timer = cv2.getTickCount()
    ok, bbox = tracker.update(frame)
    #print('tracker status: ',ok)
    fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);
    if ok:
      p1 = (int(bbox[0]), int(bbox[1]))
      p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
      cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
    else :
      # Tracking failure
      cv2.putText(frame, "Tracking failure detected", (100,80), 
      cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
      cv2.putText(frame, trackerType + " Tracker", (100,20), 
      cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2);
  
    # Display FPS on frame
    cv2.putText(frame, "FPS : " + str(int(fps)), (100,50), 
    cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2);
  
    # Display result
    out.write(frame)
out.release()
cap.release()
print('Traking achieved')

Traking achieved


In [66]:
# define helper function to display videos
import io 
from IPython.display import HTML
from base64 import b64encode
def show_video(file_name, width=640):
  # show resulting deepsort video
  mp4 = open(file_name,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width="{0}" controls>
        <source src="{1}" type="video/mp4">
  </video>
  """.format(width, data_url))

In [67]:
# convert resulting video from avi to mp4 file format
import os
!rm output2.mp4
#path_video = os.path.join("outputs","tracker.avi")
#%cd outputs/
!ffmpeg -y -loglevel panic -i output.mp4 output2.mp4
#%cd ..

# output object tracking video
#path_output = os.path.join("outputs","output.mp4")
#!ls -al
show_video('output2.mp4', width=640)